In [31]:
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

import os

import numpy as np
import tensorflow as tf
import deepchem as dc
from deepchem.models.tensorgraph.models.graph_models import GraphConvTensorGraph
from deepchem.utils.save import load_from_disk
from deepchem.feat.graph_features import ConvMolFeaturizer, WeaveFeaturizer

input_data='./ChEMBL/hDAT_pIC50.csv'

tasks = ['affinity']
featurizer=WeaveFeaturizer()

loader = dc.data.CSVLoader(tasks=tasks, smiles_field="canonical_smiles",featurizer=featurizer)
dataset=loader.featurize(input_data)


Loading raw samples now.
shard_size: 8192
About to start loading CSV from ./ChEMBL/hDAT_pIC50.csv
Loading shard 1 of size 8192.
Featurizing sample 0
Featurizing sample 1000
TIMING: featurizing shard 0 took 4.841 s
TIMING: dataset construction took 5.527 s
Loading dataset from disk.


In [34]:
import deepchem as dc
import tempfile, shutil
from deepchem.splits.splitters import IndexSplitter

splitter=IndexSplitter()
train_data,valid_data,test_data=splitter.train_valid_test_split(dataset)



len(train_data),len(valid_data),len(test_data)

print(train_data)

TIMING: dataset construction took 0.713 s
Loading dataset from disk.
TIMING: dataset construction took 0.279 s
Loading dataset from disk.
TIMING: dataset construction took 0.286 s
Loading dataset from disk.


In [37]:
from deepchem.models.tensorgraph.models.graph_models import GraphConvTensorGraph, WeaveTensorGraph

#model = GraphConvTensorGraph(len(tasks), dropout=0.5, batch_size=50, mode='regression',tensorboard=True, tensorboard_log_frequency=1, model_dir='./')
model = WeaveTensorGraph(len(tasks), dropout=0.5, batch_size=50, mode='regression')


model.fit(train_data, nb_epoch=100)

from sklearn.metrics import mean_absolute_error
from deepchem.metrics import rms_score
import deepchem.hyper.grid_search

metric = dc.metrics.Metric(
    mean_absolute_error, task_averager=np.mean, mode="regression")

print("Evaluating model")
train_scores = model.evaluate(train_data, [metric])

valid_scores = model.evaluate(valid_data, [metric])


Starting epoch 0
Starting epoch 1
Starting epoch 2
Starting epoch 3
Starting epoch 4
Starting epoch 5
Starting epoch 6
Starting epoch 7
Starting epoch 8
Starting epoch 9
Starting epoch 10
Starting epoch 11
Starting epoch 12
Starting epoch 13
Starting epoch 14
Starting epoch 15
Starting epoch 16
Starting epoch 17
Starting epoch 18
Starting epoch 19
Starting epoch 20
Starting epoch 21
Starting epoch 22
Starting epoch 23
Starting epoch 24
Starting epoch 25
Starting epoch 26
Starting epoch 27
Starting epoch 28
Starting epoch 29
Starting epoch 30
Starting epoch 31
Starting epoch 32
Starting epoch 33
Starting epoch 34
Starting epoch 35
Starting epoch 36
Starting epoch 37
Starting epoch 38
Starting epoch 39
Starting epoch 40
Starting epoch 41
Starting epoch 42
Starting epoch 43
Starting epoch 44
Starting epoch 45
Starting epoch 46
Starting epoch 47
Starting epoch 48
Starting epoch 49
Starting epoch 50
Starting epoch 51
Starting epoch 52
Starting epoch 53
Starting epoch 54
Starting epoch 55
St

Evaluating model


NameError: name 'labels' is not defined

In [4]:


metric = dc.metrics.Metric(
    dc.metrics.roc_auc_score, np.mean, mode="classification")

def reshape_y_pred(y_true, y_pred):
    """
    TensorGraph.Predict returns a list of arrays, one for each output
    We also have to remove the padding on the last batch
    Metrics taks results of shape (samples, n_task, prob_of_class)
    """
    n_samples = len(y_true)
    retval = np.stack(y_pred, axis=1)
    return retval[:n_samples]


print("Evaluating model")
train_predictions = tg.predict_on_generator(data_generator(train_dataset, predict=True))
train_predictions = reshape_y_pred(train_dataset.y, train_predictions)
train_scores = metric.compute_metric(train_dataset.y, train_predictions, train_dataset.w)
print("Training ROC-AUC Score: %f" % train_scores)

valid_predictions = tg.predict_on_generator(data_generator(valid_dataset, predict=True))
valid_predictions = reshape_y_pred(valid_dataset.y, valid_predictions)
valid_scores = metric.compute_metric(valid_dataset.y, valid_predictions, valid_dataset.w)
print("Valid ROC-AUC Score: %f" % valid_scores)

Evaluating model
computed_metrics: [0.9997852227270151, 0.9997641698779705, 0.9980244481841283, 0.9995202993637755, 0.9967420976613437, 0.9995675472073571, 0.9998068417071155, 0.99649519218699, 0.999473667726871, 0.9994320212341481, 0.9966406593169451, 0.9993969905465123]
Training ROC-AUC Score: 0.998721
computed_metrics: [0.8059169841191312, 0.84744957010582, 0.8510217084603832, 0.8047311896288765, 0.6665568181818182, 0.7481807096215876, 0.7356135340948425, 0.8284365532304657, 0.8275724040499175, 0.7689792340884574, 0.8947519701070685, 0.8356696813092164]
Validation ROC-AUC Score: 0.801240
